In [1]:
import json
import numpy
import operator

In [92]:
ls ../../data/pure_agg_gt/saved_dictionaries/

norm_pure_agg_gt_values.json  pure_agg_gt_quest2ads.json
pure_agg_gt_answer_key.json


In [82]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        # skip over aggregate single answer
        # this method made more complicated due
        # to inconsistent submission format
        if len(entry) == expected_length:
            if entry[index_of_score] > i:
                print "ORDERING PROBLEM"
                return None
            i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        # skip over aggregate single answer
        # this method made more complicated due
        # to inconsistent submission format
        if len(entry) == expected_length:
            if entry[index_of_ad] not in seen:
                uniqs.append(entry)
                seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [51]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [182]:
def AggMetric(gt_answ, sub_answ, feature):
    function = pure_agg_metric_params[feature]['function']
    amount = pure_agg_metric_params[feature]['amount']
    
    if feature in ['price', 'height', 'weight', 'age']:
        sub_answ = float(sub_answ)
        
        # Allow for centimeter height
        if feature == 'height':
            if sub_answ > 100:
                sub_answ *= 0.393701
                
        # Allow for kilogram weight
        if feature == 'weight':
            if sub_answ < 90:
                sub_answ *= 2.20462
                
        diff = gt_answ - sub_answ
        if function == 'multiply':
            if numpy.absolute(diff) <= (gt_answ * amount):
                metric = 1
            else:
                metric = 0
        elif function == 'add':
            if numpy.absolute(diff) <= amount:
                metric = 1
            else:
                metric = 0
    if function == 'match':
        if gt_answ in sub_answ.lower():
            metric = 1
        else:
            metric = 0
            
    return metric

In [183]:
pure_agg_submission_files = {}
pure_agg_submission_files['Georgetown'] = '../../data/team_submissions/Georgetown/GroundTruth/pure_agg_answer.json'
pure_agg_submission_files['Uncharted'] = '../../data/team_submissions/Uncharted/GroundTruth/uncharted_GT_pure_aggregate_answers_JSON_LINES.json'
pure_agg_submission_files['ISI'] = '../../data/team_submissions/ISI/GroundTruth/parsed_pure_aggregate_gt_v3_all_answers.jl'

gt_file_path = '../../data/pure_agg_gt/saved_dictionaries/'
gt_docs_file = gt_file_path + 'pure_agg_gt_quest2ads.json'
gt_agg_file = gt_file_path + 'pure_agg_gt_answer_key.json'

agg_parameters_file = gt_file_path + 'pure_agg_metric_parameters.json'

## Load Submissions

In [184]:
Submissions = {}

for team in pure_agg_submission_files.keys():
    pf_sub_file = pure_agg_submission_files[team]
    subs = []
    with open(pf_sub_file, 'r') as f:
        for line in f:
            temp = json.loads(line)
            subs.append(temp)
    Submissions[team] = subs
    del subs

## Load GT

In [185]:
f = open(gt_docs_file, 'r')
pure_agg_gt = eval(f.read())
f.close()

f = open(gt_agg_file, 'r')
pure_agg_answ_gt = eval(f.read())
f.close()

f = open(agg_parameters_file, 'r')
pure_agg_metric_params = eval(f.read())
f.close()

In [186]:
NewEval = {}

## Create Dictionary

In [190]:
print 'When checking manually, note the implicit unit conversions performed in AggMetric()'

for team in Submissions.keys():

    NewEval['Pure Aggregate'] = {}
    
    sub_data = Submissions[team]
    gt_data = pure_agg_gt

    try:
        del submissions
    except:
        pass
    for sub in sub_data:
        submissions = clean_sub(sub['answers'], 1, 2, 3)

        q_id = sub['question_id']
        gt = gt_data[q_id]
        NewEval['Pure Aggregate'][q_id] = {}
        NewEval['Pure Aggregate'][q_id]['avg_precision'] = AvgPrecision(gt, submissions, 1, 2)
        
        # Calculate Aggregate Metric
        if len(sub['aggregate']) == 1:
            metric = AggMetric(pure_agg_answ_gt[q_id]['agg_answer'], sub['aggregate'][0], pure_agg_answ_gt[q_id]['feature'])
        elif len(sub['aggregate']) > 1:
            print "MULTIPLE AGG SUBMISSIONS"
        elif len(sub['aggregate']) == 0:
            metric = 0
        NewEval['Pure Aggregate'][q_id]['agg_metric'] = metric
    
    # Manually check matching
    print ' '
    print team.upper()
    print ' '  
    for entry in Submissions[team]:
        q_id = entry['question_id']
        sub_agg = entry['aggregate']
        gt_agg = pure_agg_answ_gt[q_id]['agg_answer']
        points = NewEval['Pure Aggregate'][q_id]['agg_metric']
        print '{0} {1} {2}'.format(sub_agg, gt_agg, points)
        
    output_file = team + '/' + team + '_pure_agg_gt_results_rewrite.json'
    with open(output_file, 'w') as outfile:
        json.dump(NewEval, outfile, indent=2)

When checking manually, note the implicit unit conversions performed in AggMetric()
 
GEORGETOWN
 
[75] 70.0 1
[u'Mizz'] anna 0
[u'5149091699'] 8137247050 0
[19] 19 1
[u'120'] 120 1
[u'1'] 60 0
[] 125 0
[u'110'] 110 1
[160] 63 1
[28] 28 1
[u'Churchland'] paige 0
[u'Vanessa'] maria 0
[u'black'] blonde 0
[u'Savannah'] fort wayne 0
[u'asian'] asian 1
 
ISI
 
[u'maria'] maria 1
[u'140.054418273'] 70.0 0
[u'22'] 19 0
[u'brooklyn'] anna 0
[u'8137247050'] 8137247050 1
[u'103.314662334'] 120 0
[u'60.0'] 60 1
[u'56.0'] 125 1
[u'49.0'] 110 1
[u'160.003341795'] 63 1
[u'june'] paige 0
[u'asian'] asian 1
[u''] blonde 0
[u'28'] 28 1
[u'fort wayne'] fort wayne 1
 
UNCHARTED
 
[u'maria'] maria 1
[127.12121212121212] 70.0 0
[19] 19 1
[u'jessica'] anna 0
[u'7865064729'] 8137247050 0
[120] 120 1
[60] 60 1
[125] 125 1
[110] 110 1
[63] 63 1
[u'paige'] paige 1
[u'asian'] asian 1
[u'brunette'] blonde 0
[28] 28 1
[u'Savannah,GA,USA'] fort wayne 0


## Save Evaluation Dict

In [ ]:
save_file = extractions +'.json'
with open(save_file, 'w') as outfile:
    json.dump(NewEval, outfile, indent=2)

In [163]:
ls 

Georgetown/             Pure_Agg_GT_Eval.ipynb
ISI/                    Uncharted/


In [181]:
news = 3 
news *= 2
news

6